In [ ]:
import pandas as pd

df = pd.read_csv('data/data4.tsv', delimiter = '\t')

df

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([ img_to_array(load_img(i)) for i in df['image'].values ])

imgs.shape

In [ ]:
import cv2

cols = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8']

w = imgs.shape[2]
h = imgs.shape[1]

dv = df[cols].astype(np.float)

for i in range(0, len(cols), 2):
    dv[cols[i]] /= w
    dv[cols[i + 1]] /= h

labels = dv.values

labels.shape

In [ ]:
labels[0]

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, Reshape, GRU, Conv2D, MaxPool2D, Permute, LeakyReLU, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional, TimeDistributed

input = Input(shape = imgs.shape[1:])

x = input

x = BatchNormalization()(x)

x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

sh = (int(x.shape[1]), int(x.shape[2]))

x = Conv2D(1, 1, activation = 'relu')(x)

x = Reshape(sh)(x)

x1 = x
x1 = Bidirectional(GRU(128, return_sequences = True, activation = 'relu'))(x1)
x1 = TimeDistributed(Dense(sh[1] * 64))(x1)
x1 = TimeDistributed(Dense(sh[1] * 64))(x1)
x1 = LeakyReLU()(x1)
x1 = Reshape(sh + (-1,))(x1)

x2 = Permute((2, 1))(x)
x2 = Bidirectional(GRU(128, return_sequences = True, activation = 'relu'))(x2)
x2 = TimeDistributed(Dense(sh[0] * 64))(x2)
x2 = TimeDistributed(Dense(sh[0] * 64))(x2)
x2 = LeakyReLU()(x2)
x2 = Permute((2, 1))(x2)
x2 = Reshape(sh + (-1,))(x2)

x = concatenate([x1, x2])

x = Flatten()(x)

x = Dense(512, activation = 'relu')(x)
x = Dense(512, activation = 'relu')(x)

x = Dropout(0.3)(x)

output = Dense(labels.shape[-1], activation = 'sigmoid')(x)

model = Model(inputs = input, outputs = output)

model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])

model.summary()

In [ ]:
hist = model.fit(imgs, labels, initial_epoch = 0, epochs = 100, batch_size = 10)
hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8.0, 4.0)

plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])

plt.subplot(1, 2, 2)
plt.plot(hist.history['mean_absolute_error'])


In [ ]:
model.save('model/cnn-rnn-1_0.h5')

In [ ]:
def scale(d):
    for i in range(0, len(d), 2):
        d[i] *= w
        d[i + 1] *= h
        
    return d

def predict(i, s = 6.0):
    plt.rcParams['figure.figsize'] = (s, s)

    img = imgs[i]
    
    r = scale(np.copy(labels[i])).astype(int)
    p = scale(model.predict(np.array([img]))[0]).astype(int)
   
    print(r)
    print(p)
    
    line_size = 3
    
    r_color = (0, 255, 0)
    p_color = (255, 0, 0)

    img = img.astype(int)
    
    for i in range(0, len(r), 4):
        img = cv2.line(img, (r[i], r[i + 1]), (r[i + 2], r[i + 3]), r_color, line_size)
        img = cv2.line(img, (p[i], p[i + 1]), (p[i + 2], p[i + 3]), p_color, line_size)
                       
    plt.imshow(img)
    

In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)

In [ ]:
predict(4)